##IMPORTING LIBRARIES

In [ ]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

In [ ]:
# Parameters for synthetic dataset
num_classes = 2
num_images = 100  # total images per class
image_size = (224, 224)  # image size

In [ ]:
# Create directories
base_dir = 'synthetic_data'
os.makedirs(os.path.join(base_dir, 'train', 'class1'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'train', 'class2'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'val', 'class1'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'val', 'class2'), exist_ok=True)


In [ ]:
# Generate synthetic images
for cls in range(num_classes):
    for i in range(num_images):
        # Create a blank image and fill it with random colors
        image = np.random.randint(255, size=(image_size[0], image_size[1], 3), dtype=np.uint8)
        # Save the image
        cv2.imwrite(os.path.join(base_dir, 'train', f'class{cls + 1}', f'image_{i}.jpg'), image)
        # For validation, save a subset of images
        if i < 10:  # Change this number for more validation images
            cv2.imwrite(os.path.join(base_dir, 'val', f'class{cls + 1}', f'image_{i}.jpg'), image)

print("Synthetic dataset created.")


Synthetic dataset created.


In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Preprocess the data
x_train, x_test = x_train.astype('float32') / 255.0, x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
# Create ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2,
                                    height_shift_range=0.2, shear_range=0.2,
                                    zoom_range=0.2, horizontal_flip=True)

In [ ]:
def load_pretrained_model(model_name='ResNet50', input_shape=(224, 224, 3), num_classes=2):
    if model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Only ResNet50 and VGG16 are supported.")

    # Freeze all layers in the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom dense layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Create the new model
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [ ]:
# For CIFAR-10
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)

# For synthetic dataset
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('synthetic_data/train/', target_size=(224, 224),
                                                    batch_size=32, class_mode='categorical')
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory('synthetic_data/val/', target_size=(224, 224),
                                                batch_size=32, class_mode='categorical')

Found 200 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [ ]:
# Load the model (ResNet50 or VGG16)
model = load_pretrained_model(model_name='ResNet50', input_shape=(224, 224, 3), num_classes=2)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 54s 6s/step - accuracy: 0.4964 - loss: 1.1085 - val_accuracy: 0.5000 - val_loss: 0.8418
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 43s 6s/step - accuracy: 0.5115 - loss: 0.8717 - val_accuracy: 0.5000 - val_loss: 0.7240
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 41s 6s/step - accuracy: 0.5820 - loss: 0.7926 - val_accuracy: 0.5000 - val_loss: 0.6975
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 41s 6s/step - accuracy: 0.4607 - loss: 0.8745 - val_accuracy: 0.5000 - val_loss: 0.7191
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 41s 7s/step - accuracy: 0.5991 - loss: 0.7575 - val_accuracy: 0.5000 - val_loss: 0.6985
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 41s 6s/step - accuracy: 0.4983 - loss: 0.7534 - val_accuracy: 0.5000 - val_loss: 0.7042
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 82s 6s/step - accuracy: 0.5369 - loss: 0.7692 - val_accuracy: 0.5000 - val_loss: 0.7050
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 41s 6s/step - accuracy: 0.4798 - loss: 0.7882 - val_accuracy: 0.5000 - val_loss: 0.7479
Epoch 9/10
7/7 ━━━━

###EVALUATING MODEL

In [ ]:
# Evaluate on the test data
test_loss, test_accuracy = model.evaluate(val_generator)  # Use validation data for evaluation
print(f"Test Accuracy: {test_accuracy:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5000 - loss: 0.7348
Test Accuracy: 0.5000


In [ ]:
# Predictions for precision, recall, and classification report
y_pred = model.predict(val_generator)
y_pred_classes = y_pred.argmax(axis=-1)
y_true = val_generator.classes

print(classification_report(y_true, y_pred_classes, target_names=val_generator.class_indices.keys()))

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
              precision    recall  f1-score   support

      class1       0.50      1.00      0.67        10
      class2       0.00      0.00      0.00        10

    accuracy                           0.50        20
   macro avg       0.25      0.50      0.33        20
weighted avg       0.25      0.50      0.33        20



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Assuming you have already trained your model and have the validation data
# Use val_generator to get the evaluation metrics

# Evaluate on the validation data
test_loss, test_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {test_loss:.4f}")
print(f"Validation Accuracy: {test_accuracy:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.5000 - loss: 0.7348
Validation Loss: 0.7348
Validation Accuracy: 0.5000


In [ ]:
# Predictions for precision, recall, and classification report
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the predicted class indices
y_true = val_generator.classes  # Get the true class indices

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [ ]:
# Generate classification report
from sklearn.metrics import classification_report

In [ ]:
# Get class names
class_names = val_generator.class_indices.keys()

In [ ]:
# Print the classification report
report = classification_report(y_true, y_pred_classes, target_names=class_names)
print(report)

              precision    recall  f1-score   support

      class1       0.50      1.00      0.67        10
      class2       0.00      0.00      0.00        10

    accuracy                           0.50        20
   macro avg       0.25      0.50      0.33        20
weighted avg       0.25      0.50      0.33        20



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
